In [1]:
# importing packages
from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# loforest and locart functions
from CP2LFI.loforest import ConformalLoforest
from CP2LFI.scores import LambdaScore

from clover import Scores
from clover import LocartSplit

from copy import deepcopy
from tqdm import tqdm

from scipy import stats
from scipy.optimize import minimize_scalar

import time

\section{Comparing our methods with quantile regression in a GMM setting}
Taking $\theta \in \Theta = [0,5]$ we consider a one-dimensional Normal mixture with unknown mean but known unit variance:
\begin{align*}
X \sim 0.5 N(\theta, 1) + 0.5 N(-\theta, 1)
\end{align*}
We consider three different sample sizes:
$$ N = 10, 100, 1000 $$


In [2]:
# simulator
def sim_gmm(n, theta):
    group = np.random.binomial(n=1, p=0.5, size=n)
    X = ((group == 0) * (np.random.normal(theta, 1, size=n))) + (
        (group == 1) * (np.random.normal(-theta, 1, size=n))
    )
    return X


# randomly sampling from gmm
def sample_gmm(n, N, seed=450):
    np.random.seed(seed)
    thetas = np.random.uniform(0, 5, size=n)
    lambdas = np.zeros(n)
    i = 0
    for theta in tqdm(thetas, desc="Simulating lambdas and thetas"):
        X = sim_gmm(N, theta)
        lambdas[i] = compute_lrt_statistic(theta, X)
        i += 1
    return thetas, lambdas


# likelihood function
def l_func(theta, x):
    # prob from X
    p_x = np.log(
        (0.5 * stats.norm.pdf(x, loc=theta, scale=1))
        + (0.5 * stats.norm.pdf(x, loc=-theta, scale=1))
    )
    return -(np.sum(p_x))


# likelihood ratio statistic
def compute_lrt_statistic(theta_0, X, lower=0, upper=5):
    # computing MLE by grid
    res = minimize_scalar(l_func, args=(X), bounds=(0, 5))
    mle_theta = res.x
    lrt_stat = -2 * ((-l_func(theta_0, X)) - (-l_func(mle_theta, X)))
    return lrt_stat


# naive method
def naive(alpha, B=1000, N=100, lower=0, upper=5, seed=250, naive_n=100):
    np.random.seed(seed)
    n_grid = int(B / naive_n)
    thetas = np.linspace(lower, upper, n_grid)
    quantiles = {}
    for theta in tqdm(thetas, desc="Obtaining naive quantiles"):
        LRT_stat = np.zeros(B)
        for i in range(0, B):
            X = sim_gmm(N, theta)
            LRT_stat[i] = compute_lrt_statistic(theta, X)
        quantiles[theta] = np.quantile(LRT_stat, q=1 - alpha)
    return quantiles


# naive predict function
def predict_naive_quantile(theta_grid, quantiles_dict):
    thetas_values = np.array(list(quantiles_dict.keys()))
    quantiles_list = []
    for theta in theta_grid:
        idx = thetas_values[int(np.argmin(np.abs(theta - thetas_values)))]
        quantiles_list.append(quantiles_dict[idx])
    return quantiles_list

Function to evaluate coverage for a given sample size $N$:


In [3]:
def evaluate_coverage(quantiles_dict, thetas, alpha=0.05, n=500, N=100):
    err_data = np.zeros((thetas.shape[0], 4))
    coverage_data = np.zeros((thetas.shape[0], 4))
    j = 0
    for theta in tqdm(thetas, desc="Computing coverage for each method"):
        # generating several lambdas
        LRT_stat = np.zeros(n)
        for i in range(0, n):
            samples = sim_gmm(N, theta)
            LRT_stat[i] = compute_lrt_statistic(theta, samples)

        # comparing coverage of methods
        locart_cover = np.mean(LRT_stat <= quantiles_dict["locart"][j])
        loforest_cover = np.mean(LRT_stat <= quantiles_dict["loforest"][j])
        boosting_cover = np.mean(LRT_stat <= quantiles_dict["boosting"][j])
        naive_cover = np.mean(LRT_stat <= quantiles_dict["naive"][j])

        # appending the errors
        err_locart = np.abs(locart_cover - (1 - alpha))
        err_loforest = np.abs(loforest_cover - (1 - alpha))
        err_boosting = np.abs(boosting_cover - (1 - alpha))
        err_naive = np.abs(naive_cover - (1 - alpha))

        # saving in numpy array
        err_data[j, :] = np.array([err_locart, err_loforest, err_boosting, err_naive])
        coverage_data[j, :] = np.array(
            [locart_cover, loforest_cover, boosting_cover, naive_cover]
        )
        j += 1

    # obtaining MAE and standard error for each method
    mae_vector, std_vector = np.mean(err_data, axis=0), np.std(err_data, axis=0)
    stats_data = pd.DataFrame(
        {
            "methods": ["LOCART", "LOFOREST", "boosting", "naive"],
            "MAE": mae_vector,
            "str": std_vector / (np.sqrt(thetas.shape[0])),
        }
    )

    coverage_data = pd.DataFrame(
        {
            "thetas": thetas,
            "LOCART": coverage_data[:, 0],
            "LOFOREST": coverage_data[:, 1],
            "boosting": coverage_data[:, 2],
            "naive": coverage_data[:, 3],
        }
    )
    return [stats_data, coverage_data]

Now, we compare all methods fixing $\alpha = 0.05$, $N = 100$ and $B = 1000$ for locart, loforest and boosting and n_grid = 30 for naive:


In [4]:
# fixing parameters
N, B, alpha = 100, 1000, 0.05

Starting by fitting naive:

In [5]:
# running naive
start_time = time.time()
naive_quantiles = naive(alpha, B = B, N = N)
end_time = time.time()

running_time = end_time - start_time
print(f"Naive running time: {running_time} seconds.")

Obtaining naive quantiles:   0%|          | 0/10 [00:00<?, ?it/s]

Obtaining naive quantiles:  10%|█         | 1/10 [00:02<00:24,  2.69s/it]

Obtaining naive quantiles:  20%|██        | 2/10 [00:04<00:18,  2.26s/it]

Obtaining naive quantiles:  30%|███       | 3/10 [00:06<00:13,  1.92s/it]

Obtaining naive quantiles:  40%|████      | 4/10 [00:07<00:10,  1.73s/it]

...

Obtaining naive quantiles:  80%|████████  | 8/10 [00:12<00:02,  1.26s/it]

Obtaining naive quantiles:  90%|█████████ | 9/10 [00:13<00:01,  1.19s/it]

Obtaining naive quantiles: 100%|██████████| 10/10 [00:15<00:00,  1.59s/it]

Obtaining naive quantiles: 100%|██████████| 10/10 [00:15<00:00,  1.58s/it]

Naive running time: 15.812763452529907 seconds.


Sampling $\theta$ and $\lambda$ to fit the models:


In [6]:
thetas, model_lambdas = sample_gmm(n = B, N = N, seed = 45)
model_thetas = thetas.reshape(-1, 1)

Simulating lambdas and thetas:   0%|          | 0/1000 [00:00<?, ?it/s]

Simulating lambdas and thetas:   8%|▊         | 75/1000 [00:00<00:01, 742.39it/s]

Simulating lambdas and thetas:  15%|█▌        | 150/1000 [00:00<00:01, 725.76it/s]

Simulating lambdas and thetas:  22%|██▎       | 225/1000 [00:00<00:01, 734.76it/s]

Simulating lambdas and thetas:  30%|██▉       | 299/1000 [00:00<00:00, 729.44it/s]

...

Simulating lambdas and thetas:  74%|███████▍  | 740/1000 [00:01<00:00, 703.32it/s]

Simulating lambdas and thetas:  82%|████████▏ | 817/1000 [00:01<00:00, 722.13it/s]

Simulating lambdas and thetas:  89%|████████▉ | 890/1000 [00:01<00:00, 715.13it/s]

Simulating lambdas and thetas:  96%|█████████▌| 962/1000 [00:01<00:00, 714.72it/s]

Simulating lambdas and thetas: 100%|██████████| 1000/1000 [00:01<00:00, 721.57it/s]

Fitting all other methods:

In [7]:
# locart quantiles
start_time = time.time()
locart_object = LocartSplit(LambdaScore, None, alpha = alpha, is_fitted = True, split_calib = False)
locart_quantiles = locart_object.calib(model_thetas, model_lambdas, min_samples_leaf = 100)
end_time = time.time()
print(f"LOCART running time: {end_time - start_time} seconds.")

# loforest quantiles
start_time = time.time()
loforest_object = ConformalLoforest(LambdaScore, None, alpha = alpha, is_fitted = True, split_calib = False)
loforest_object.calibrate(model_thetas, model_lambdas, min_samples_leaf = 300)
end_time = time.time()
print(f"LOFOREST running time: {end_time - start_time} seconds.")

# boosting quantiles
start_time = time.time()
model = HistGradientBoostingRegressor(loss="quantile", max_iter = 1000,
quantile = 1 - alpha, random_state = 105, n_iter_no_change = 20, early_stopping = True)
model.fit(model_thetas, model_lambdas)
end_time = time.time()
print(f"Boosting running time: {end_time - start_time} seconds.")

LOCART running time: 0.011343717575073242 seconds.
LOFOREST running time: 0.11741161346435547 seconds.


Boosting running time: 0.21065115928649902 seconds.


Predicting for $theta$ in a relatively thin grid:


In [8]:
n_out = 300
thetas = np.linspace(0.001, 4.999, n_out)

# naive quantiles
naive_list = predict_naive_quantile(thetas, naive_quantiles)

# locart quantiles
idxs = locart_object.cart.apply(thetas.reshape(-1, 1))
list_locart_quantiles = [locart_quantiles[idx] for idx in idxs]

# loforest
loforest_cutoffs = loforest_object.compute_cutoffs(thetas.reshape(-1, 1))

# boosting
boosting_quantiles = model.predict(thetas.reshape(-1, 1))

# dictionary of quantiles
quantile_dict = {
    "naive": naive_list,
    "locart": list_locart_quantiles,
    "loforest": loforest_cutoffs,
    "boosting": boosting_quantiles,
}

Obtaining coverage data for $N = 100$:

In [9]:
cover_data = evaluate_coverage(quantile_dict, thetas, n = 1000)

Computing coverage for each method:   0%|          | 0/300 [00:00<?, ?it/s]

Computing coverage for each method:   0%|          | 1/300 [00:02<12:28,  2.50s/it]

Computing coverage for each method:   1%|          | 2/300 [00:04<12:24,  2.50s/it]

Computing coverage for each method:   1%|          | 3/300 [00:07<12:32,  2.53s/it]

Computing coverage for each method:   1%|▏         | 4/300 [00:10<12:42,  2.57s/it]

...

Computing coverage for each method:  99%|█████████▉| 297/300 [06:46<00:05,  1.73s/it]

Computing coverage for each method:  99%|█████████▉| 298/300 [06:48<00:03,  1.85s/it]

Computing coverage for each method: 100%|█████████▉| 299/300 [06:50<00:02,  2.01s/it]

Computing coverage for each method: 100%|██████████| 300/300 [06:53<00:00,  2.18s/it]

Computing coverage for each method: 100%|██████████| 300/300 [06:53<00:00,  1.38s/it]

Analising MAE and standard errors:

In [10]:
#| echo: false
keys = cover_data[0].iloc[:, 0].values
mae = cover_data[0].iloc[:, 1].values
std_err = cover_data[0].iloc[:, 2].values

plt.figure(figsize=(10, 6))
plt.errorbar(keys, mae, yerr=std_err, fmt='o')
plt.xlabel('Method')
plt.ylabel('Error')
plt.title('Coverage MAE error bars for random thetas and lambdas used in training')
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
plt.show()

<Figure size 3000x1800 with 1 Axes>

Visualizing coverage as a function of $x$:

In [11]:
#| echo: false
# Melt the coverage data
coverage_data_melted = pd.melt(
    cover_data[1], id_vars=["thetas"], var_name="method", value_name="coverage")
# Plot the coverage as function of the thetas generated
g = sns.FacetGrid(coverage_data_melted, col="method", hue = "method",
 col_wrap = 2, height=8, aspect=1.10, palette = "Set1")
g.map(sns.lineplot, "thetas", "coverage")
g.add_legend()
g.set_titles("{col_name}")
g.set_xlabels(r"$\theta$")
g.set_ylabels("Coverage")
plt.show()

/home/kuben45/anaconda3/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



<Figure size 5648.88x4800 with 4 Axes>

Now, we compute the coverage and MAE's for each method across three sample sizes $N$, through the following functions:

In [12]:
# function to compute all quantiles in a theta grid
def obtain_quantiles(
    thetas, N, B=1000, alpha=0.05, naive_seed=45, min_samples_leaf=100, naive_n=500
):
    # fitting and predicting naive
    naive_quantiles = naive(alpha=alpha, B=B, N=N, naive_n=naive_n)
    naive_list = predict_naive_quantile(thetas, naive_quantiles)

    # simulating to fit models
    theta_sim, model_lambdas = sample_gmm(n=B, N=N, seed=45)
    model_thetas = theta_sim.reshape(-1, 1)

    locart_object = LocartSplit(
        LambdaScore, None, alpha=alpha, is_fitted=True, split_calib=False
    )
    locart_quantiles = locart_object.calib(
        model_thetas, model_lambdas, min_samples_leaf=min_samples_leaf
    )

    # loforest quantiles
    loforest_object = ConformalLoforest(
        LambdaScore, None, alpha=alpha, is_fitted=True, split_calib=False
    )
    loforest_object.calibrate(
        model_thetas, model_lambdas, min_samples_leaf=min_samples_leaf
    )

    # boosting quantiles
    model = HistGradientBoostingRegressor(
        loss="quantile",
        max_iter=100,
        max_depth=3,
        quantile=1 - alpha,
        random_state=105,
        n_iter_no_change=15,
        early_stopping=True,
    )
    model.fit(model_thetas, model_lambdas)

    # naive quantiles
    naive_list = predict_naive_quantile(thetas, naive_quantiles)

    # locart quantiles
    idxs = locart_object.cart.apply(thetas.reshape(-1, 1))
    list_locart_quantiles = [locart_quantiles[idx] for idx in idxs]

    # loforest
    loforest_cutoffs = loforest_object.compute_cutoffs(thetas.reshape(-1, 1))

    # boosting
    boosting_quantiles = model.predict(thetas.reshape(-1, 1))

    # dictionary of quantiles
    quantile_dict = {
        "naive": naive_list,
        "locart": list_locart_quantiles,
        "loforest": loforest_cutoffs,
        "boosting": boosting_quantiles,
    }

    return quantile_dict


# evaluate coverage for several N's and B = 1000
def evaluate_coverage_N(
    thetas,
    N=np.array([10, 100, 1000]),
    B=1000,
    alpha=0.05,
    n=1000,
    seed=45,
    min_samples_leaf=100,
    naive_n=100,
):
    coverage_data = np.zeros((thetas.shape[0] * N.shape[0], 4))
    N_list = []
    N_list_cover = []
    methods_list = []
    np.random.seed(seed)
    seeds = np.random.choice(np.arange(0, 10**4, 1), N.shape[0], replace=False)
    k = 0
    j = 0
    for N_fixed in tqdm(N, desc="Computing coverage for each N"):
        # computing all quantiles for fixed N
        quantiles_dict = obtain_quantiles(
            thetas,
            N=N_fixed,
            B=B,
            alpha=alpha,
            naive_seed=seeds[k],
            min_samples_leaf=min_samples_leaf,
            naive_n=naive_n,
        )
        err_data = np.zeros((thetas.shape[0], 4))
        l = 0
        for theta in tqdm(thetas, desc="Computing coverage for each method"):
            # generating several lambdas
            LRT_stat = np.zeros(n)
            for i in range(0, n):
                samples = sim_gmm(N_fixed, theta)
                LRT_stat[i] = compute_lrt_statistic(theta, samples)

            # comparing coverage of methods
            locart_cover = np.mean(LRT_stat <= quantiles_dict["locart"][l])
            loforest_cover = np.mean(LRT_stat <= quantiles_dict["loforest"][l])
            boosting_cover = np.mean(LRT_stat <= quantiles_dict["boosting"][l])
            naive_cover = np.mean(LRT_stat <= quantiles_dict["naive"][l])

            # appending the errors
            err_locart = np.abs(locart_cover - (1 - alpha))
            err_loforest = np.abs(loforest_cover - (1 - alpha))
            err_boosting = np.abs(boosting_cover - (1 - alpha))
            err_naive = np.abs(naive_cover - (1 - alpha))

            # saving in numpy array
            err_data[l, :] = np.array(
                [err_locart, err_loforest, err_boosting, err_naive]
            )

            coverage_data[j, :] = np.array(
                [locart_cover, loforest_cover, boosting_cover, naive_cover]
            )
            N_list_cover.append(N_fixed)

            j += 1
            l += 1
        methods_list.extend(["LOCART", "LOFOREST", "boosting", "naive"])
        if k == 0:
            mae_vector = np.mean(err_data, axis=0)
            std_vector = np.std(err_data, axis=0) / (np.sqrt(thetas.shape[0]))
        else:
            mean = np.mean(err_data, axis=0)
            std = np.std(err_data, axis=0) / (np.sqrt(thetas.shape[0]))
            mae_vector, std_vector = np.concatenate((mae_vector, mean)), np.concatenate(
                (std_vector, std)
            )
        k += 1
        N_list.extend([N_fixed] * 4)

    # obtaining MAE and standard error for each method
    stats_data = pd.DataFrame(
        {
            "methods": methods_list,
            "N": N_list,
            "MAE": mae_vector,
            "se": std_vector,
        }
    )

    coverage_data = pd.DataFrame(
        {
            "thetas": np.tile(thetas, N.shape[0]),
            "N": N_list_cover,
            "LOCART": coverage_data[:, 0],
            "LOFOREST": coverage_data[:, 1],
            "boosting": coverage_data[:, 2],
            "naive": coverage_data[:, 3],
        }
    )
    return [stats_data, coverage_data]

Computing coverage:

In [13]:
#| output: false
n_out = 300
thetas = np.linspace(0.001, 4.999, n_out)

coverage_data_N_1000 = evaluate_coverage_N(thetas, B = 1000, naive_n = 100, n = 1000)

Computing coverage for each N:   0%|          | 0/3 [00:00<?, ?it/s]

Obtaining naive quantiles:   0%|          | 0/10 [00:00<?, ?it/s]

Obtaining naive quantiles:  10%|█         | 1/10 [00:02<00:20,  2.24s/it]

Obtaining naive quantiles:  20%|██        | 2/10 [00:04<00:16,  2.10s/it]

Obtaining naive quantiles:  30%|███       | 3/10 [00:05<00:12,  1.82s/it]

...

Computing coverage for each method: 100%|█████████▉| 299/300 [09:23<00:02,  2.33s/it]

Computing coverage for each method: 100%|██████████| 300/300 [09:26<00:00,  2.72s/it]

Computing coverage for each method: 100%|██████████| 300/300 [09:26<00:00,  1.89s/it]


Computing coverage for each N: 100%|██████████| 3/3 [24:25<00:00, 507.87s/it]

Computing coverage for each N: 100%|██████████| 3/3 [24:25<00:00, 488.59s/it]

Plotting MAE and standard error for each method and several $N$:

In [14]:
#| echo: false
fig, ax = plt.subplots(ncols=3, figsize=(10, 6))
# data for first subplot
data_10 = coverage_data_N_1000[0].query("N == 10")
keys = data_10.iloc[:, 0].values
mae = data_10.iloc[:, 2].values
std_err = data_10.iloc[:, 3].values
ax[0].errorbar(keys, mae, yerr=std_err, fmt="o")
ax[0].set_xlabel("Method")
ax[0].set_ylabel("Error")
ax[0].set_title("Estimated MAE for N = 10")


data_100 = coverage_data_N_1000[0].query("N == 100")
keys = data_100.iloc[:, 0].values
mae = data_100.iloc[:, 2].values
std_err = data_100.iloc[:, 3].values
ax[1].errorbar(keys, mae, yerr=std_err, fmt="o")
ax[1].set_xlabel("Method")
ax[1].set_ylabel("Error")
ax[1].set_title("Estimated MAE for N = 100")


data_1000 = coverage_data_N_1000[0].query("N == 1000")
keys = data_1000.iloc[:, 0].values
mae = data_1000.iloc[:, 2].values
std_err = data_1000.iloc[:, 3].values
ax[2].errorbar(keys, mae, yerr=std_err, fmt="o")
ax[2].set_ylabel("Error")
ax[2].set_title("Estimated MAE for N = 1000")

for a in ax:
    a.set_xticklabels(a.get_xticklabels(), rotation=90)
plt.tight_layout()
plt.show()

/tmp/ipykernel_32981/3018600969.py:32: UserWarning:

FixedFormatter should only be used together with FixedLocator



<Figure size 3000x1800 with 3 Axes>

Plotting the coverage line of all methods:

In [15]:
#| echo: false
# plotting coverage
coverage_data_melted = pd.melt(
    coverage_data_N_1000[1],
    id_vars=["thetas", "N"],
    var_name="method",
    value_name="coverage",
)
# Plot the coverage as function of the thetas generated
g = sns.FacetGrid(
    coverage_data_melted,
    row="N",
    col="method",
    hue="method",
    height= 8,
    aspect=1.20,
    palette="Set1",
    margin_titles = True
)
g.map(sns.lineplot, "thetas", "coverage")
g.add_legend()
g.set_titles("{col_name}")
g.set_xlabels(r"$\theta$")
g.set_ylabels("Coverage")
plt.tight_layout()
plt.show()

/home/kuben45/anaconda3/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



/tmp/ipykernel_32981/317467592.py:24: UserWarning:

The figure layout has changed to tight



<Figure size 11888.9x7200 with 12 Axes>

Increasing $B$ to 5000 samples and testing:

In [16]:
#| output: false
coverage_data_N_5000 = evaluate_coverage_N(thetas, B=5000, naive_n=250, min_samples_leaf=300, n = 1000)

Computing coverage for each N:   0%|          | 0/3 [00:00<?, ?it/s]

Obtaining naive quantiles:   0%|          | 0/20 [00:00<?, ?it/s]

Obtaining naive quantiles:   5%|▌         | 1/20 [00:12<03:57, 12.52s/it]

Obtaining naive quantiles:  10%|█         | 2/20 [00:24<03:37, 12.09s/it]

Obtaining naive quantiles:  15%|█▌        | 3/20 [00:34<03:12, 11.34s/it]

...

Computing coverage for each method: 100%|█████████▉| 299/300 [09:32<00:02,  2.45s/it]

Computing coverage for each method: 100%|██████████| 300/300 [09:36<00:00,  2.80s/it]

Computing coverage for each method: 100%|██████████| 300/300 [09:36<00:00,  1.92s/it]


Computing coverage for each N: 100%|██████████| 3/3 [33:21<00:00, 690.15s/it]

Computing coverage for each N: 100%|██████████| 3/3 [33:21<00:00, 667.22s/it]

Plotting all results again:

In [17]:
#| echo: false
fig, ax = plt.subplots(ncols = 3, figsize=(10, 6))
# data for first subplot
data_10 = coverage_data_N_5000[0].query('N == 10')
keys = data_10.iloc[:, 0].values
mae = data_10.iloc[:, 2].values
std_err = data_10.iloc[:, 3].values
ax[0].errorbar(keys, mae, yerr = std_err, fmt = 'o')
ax[0].set_xlabel('Method')
ax[0].set_ylabel('Error')
ax[0].set_title('Estimated MAE for N = 10')


data_100 = coverage_data_N_5000[0].query('N == 100')
keys = data_100.iloc[:, 0].values
mae = data_100.iloc[:, 2].values
std_err = data_100.iloc[:, 3].values
ax[1].errorbar(keys, mae, yerr = std_err, fmt = 'o')
ax[1].set_xlabel('Method')
ax[1].set_ylabel('Error')
ax[1].set_title('Estimated MAE for N = 100')


data_1000 = coverage_data_N_5000[0].query('N == 1000')
keys = data_1000.iloc[:, 0].values
mae = data_1000.iloc[:, 2].values
std_err = data_1000.iloc[:, 3].values
ax[2].errorbar(keys, mae, yerr = std_err, fmt = 'o')
ax[2].set_xlabel('Method')
ax[2].set_ylabel('Error')
ax[2].set_title('Estimated MAE for N = 1000')

for a in ax:
    a.set_xticklabels(a.get_xticklabels(), rotation=90)
plt.tight_layout()
plt.show()

/tmp/ipykernel_32981/2204363334.py:33: UserWarning:

FixedFormatter should only be used together with FixedLocator



<Figure size 3000x1800 with 3 Axes>

In [18]:
#| echo: false
# plotting coverage
coverage_data_melted = pd.melt(
    coverage_data_N_5000[1],
    id_vars=["thetas", "N"],
    var_name="method",
    value_name="coverage",
)
# Plot the coverage as function of the thetas generated
g = sns.FacetGrid(
    coverage_data_melted,
    row="N",
    col="method",
    hue="method",
    height= 8,
    aspect=1.20,
    palette="Set1",
    margin_titles=True,
)
g.map(sns.lineplot, "thetas", "coverage")
g.add_legend()
g.set_titles("{col_name}")
g.set_xlabels(r"$\theta$")
g.set_ylabels("Coverage")
plt.tight_layout()
plt.show()

/home/kuben45/anaconda3/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



/tmp/ipykernel_32981/2342929687.py:24: UserWarning:

The figure layout has changed to tight



<Figure size 11888.9x7200 with 12 Axes>

As the naive method performed very well in all cases, lets check if the distribution of $\lambda$ is invariant across $\theta$ through histograms:

In [19]:
#| echo: false
# simulating and comparing four samples
np.random.seed(145)
lambdas_1, lambdas_2, lambdas_3, lambdas_4 = np.zeros(1000), np.zeros(1000), np.zeros(1000), np.zeros(1000)

for i in range(0, 1000):
    X = sim_gmm(1000, 0.1)
    lambdas_1[i] = compute_lrt_statistic(0.1, X)

    X = sim_gmm(1000, 1.5)
    lambdas_2[i] = compute_lrt_statistic(1.5, X)

    X = sim_gmm(1000, 3)
    lambdas_3[i] = compute_lrt_statistic(3, X)

    X = sim_gmm(1000, 4.5)
    lambdas_4[i] = compute_lrt_statistic(4.5, X)
# plotting histograms
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].hist(lambdas_1, bins=30, color='blue', alpha=0.7)
axs[0, 0].set_title(r'Histogram for $\theta = 0.1$')
axs[0, 0].set_xlabel(r'\lambda(\theta, X)')
axs[0, 0].set_ylabel('Frequency')

axs[0, 1].hist(lambdas_2, bins=30, color='red', alpha=0.7)
axs[0, 1].set_title(r'Histogram for $\theta = 1.5$')
axs[0, 1].set_xlabel(r'\lambda(\theta, X)')
axs[0, 1].set_ylabel('Frequency')

axs[1, 0].hist(lambdas_3, bins=30, color='green', alpha=0.7)
axs[1, 0].set_title(r'Histogram for $\theta = 3$')
axs[1, 0].set_xlabel(r'\lambda(\theta, X)')
axs[1, 0].set_ylabel('Frequency')

axs[1, 1].hist(lambdas_4, bins=30, color='purple', alpha=0.7)
axs[1, 1].set_title(r'Histogram for $\theta = 4.5$')
axs[1, 1].set_xlabel(r'\lambda(\theta, X)')
axs[1, 1].set_ylabel('Frequency')

Text(0, 0.5, 'Frequency')

<Figure size 3000x3000 with 4 Axes>